# Primeiro dataset

Dataset escolhido: [https://archive.ics.uci.edu/dataset/602/dry+bean+dataset](https://archive.ics.uci.edu/dataset/602/dry+bean+dataset)

## Extraindo os dados

In [64]:
import pandas as pd

path_to_data = 'data/Dry_Bean_Dataset.xlsx'

# Carregar base de dados
# DataFrame
HEADER_ROW = 0
dataset = pd.read_excel(path_to_data, sheet_name="Dry_Beans_Dataset",header=HEADER_ROW)

# DEBUG - Verificar se a base de dados foi carregada corretamente
# dataset.head(20)
# dataset.shape

## Transformando os dados

In [65]:
# Transformando as labels em valores numéricos
dataset['Class'] = dataset['Class'].astype('category').cat.codes

## Separando os dados
Seperação dos dados em features (X) e target (y)

In [66]:
NUM_OF_COLUMNS = len(dataset.columns)

y = dataset.iloc[:,-1] # extrai o target (última coluna)
X = dataset.iloc[:,0:NUM_OF_COLUMNS-1] # extrai as features (todas as colunas menos a última)

## Normalizando os dados
Deixando os valores de todas as colunas entre 0 e 1

In [67]:
from sklearn.preprocessing import MinMaxScaler

x_values = X.values
min_max_scaler = MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x_values)
X = pd.DataFrame(x_scaled)

## Criando os folds
Serão 10 folds para o k-fold cross validation

In [72]:
import numpy as np

# Transforma para array NumPy
X = np.array(X)
y = np.array(y)

# Define o número de folds
N_FOLDS = 10

## Obtendo os scores dos modelos

### Árvore de decisão

In [74]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

entropy_tree_model = DecisionTreeClassifier(criterion="entropy")
# TODO - obter também o erro médio
entropy_tree_scores = cross_val_score(entropy_tree_model, X, y, cv=N_FOLDS)

print(entropy_tree_scores)

print(f"Média de acerto da ávore de decisão com entropy: {round(np.mean(entropy_tree_scores) * 100)}%")

[0.49706314 0.73695812 0.76487877 0.90301249 0.92946363 0.91991183
 0.71418075 0.68919912 0.69213813 0.42689199]
Média de acerto da ávore de decisão com entropy: 73%
